<a href="https://colab.research.google.com/github/yuuki0416-ui/ml_text/blob/main/%E7%AC%AC6%E5%9B%9E%E6%BC%94%E7%BF%92%E8%AA%B2%E9%A1%8CPytorch_Basics_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#機械学習システム第6回課題
##62317734
##本間勇毅

# PyTorch Basics: 演習ノートブック（簡単な2クラス分類）

**PyTorchの基礎**（テンソル、`Dataset`/`DataLoader`、`nn.Module`、最適化、損失、学習ループ、評価）で学んだ内容を、**トピックを** 1つの **簡単な2クラス分類問題** に統合してまとめている。

以下は、**動作するベースライン実装** を提供しており、そのまま実行すれば、合成データの分類器が学習・評価される。

各項目にある**問**に解答しなさい。

このレポートも共通で、LLMは、何らかの形で必ず利用し、このNotebookをそのまま編集して、課題レポートを作成し、PDFとNotebookの両方を提出してください。

なお、このNotebookの実行にGPUは不要です。

## 0. セットアップ
- 依存: `torch`, `matplotlib`
- 乱数シードを固定して再現性を担保します。

In [ ]:
import math
import time
import itertools
from dataclasses import dataclass

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

def set_seed(seed: int = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # さらに厳密な再現性が必要であれば以下も（ただし速度低下の可能性）
    torch.use_deterministic_algorithms(False)
    return seed

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
SEED = set_seed(1234)
DEVICE, SEED

## 1. 合成データの生成と可視化
2次元入力 $\mathbf{x}=(x_1,x_2)$ の2クラス分類を行う

2つの “月（moons）” 形状のデータをPyTorchで生成

In [ ]:
@torch.no_grad()
def make_two_moons(n_samples=2000, noise=0.2, radius=1.0, distance=0.5):
    # 上半月
    angles1 = torch.rand(n_samples//2) * math.pi
    x1 = torch.stack([
        torch.cos(angles1),
        torch.sin(angles1)
    ], dim=1) * radius
    # 下半月（少し平行移動）
    angles2 = torch.rand(n_samples - n_samples//2) * math.pi
    x2 = torch.stack([
        1 - torch.cos(angles2),
        1 - torch.sin(angles2) - distance
    ], dim=1) * radius

    X = torch.cat([x1, x2], dim=0)
    y = torch.cat([
        torch.zeros(x1.size(0), dtype=torch.long),
        torch.ones(x2.size(0), dtype=torch.long)
    ], dim=0)
    X += noise * torch.randn_like(X)
    return X, y

X_all, y_all = make_two_moons(n_samples=3000, noise=0.25, radius=1.0, distance=0.5)
perm = torch.randperm(X_all.size(0))
X_all, y_all = X_all[perm], y_all[perm]

n_train = int(0.7 * len(X_all))
n_val = int(0.15 * len(X_all))
X_train, y_train = X_all[:n_train], y_all[:n_train]
X_val, y_val = X_all[n_train:n_train+n_val], y_all[n_train:n_train+n_val]
X_test, y_test = X_all[n_train+n_val:], y_all[n_train+n_val:]

fig = plt.figure(figsize=(5,5))
# クラスラベルで色分け、train/val/testをマーカーで区別
plt.scatter(X_train[:,0], X_train[:,1], c=y_train, s=20, marker='o', label='train', alpha=0.6, edgecolors='k', linewidth=0.3)
plt.scatter(X_val[:,0], X_val[:,1], c=y_val, s=30, marker='s', label='val', alpha=0.7, edgecolors='k', linewidth=0.3)
plt.scatter(X_test[:,0], X_test[:,1], c=y_test, s=30, marker='^', label='test', alpha=0.7, edgecolors='k', linewidth=0.3)
plt.colorbar(label='Class')
plt.legend()
plt.title('Two Moons (train/val/test)')
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

# 問1

この結果から、データは **線形分離可能** といえるか。**根拠** として図の特徴（曲線的な境界の必要性など）を簡潔に説明しなさい。

以下にセルを追加して解答しなさい。

## 2. `Dataset` と `DataLoader`
`TensorDataset` を使わず、学習用の **簡単な自作 `Dataset`** を用意します（読みやすさ重視）。

In [ ]:
class MoonsDataset(Dataset):
    def __init__(self, X: torch.Tensor, y: torch.Tensor):
        self.X = X.float()
        self.y = y.long()
    def __len__(self):
        return self.X.size(0)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 64
train_ds = MoonsDataset(X_train, y_train)
val_ds   = MoonsDataset(X_val, y_val)
test_ds  = MoonsDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size)
test_loader  = DataLoader(test_ds, batch_size=batch_size)

len(train_ds), len(val_ds), len(test_ds)

# 問2

バッチサイズを 32 / 64 / 256 に変更した場合、**1エポックの反復回数** はどのようになるか、次のセルの出力を **根拠** に、式とともに答えなさい。

以下にセルを追加して解答しなさい。

In [ ]:
def iters_per_epoch(dataset_size, batch):
    return math.ceil(dataset_size / batch)

dataset_size = len(train_ds)
for b in [32, 64, 256]:
    print(f"batch={b}: iters/epoch={iters_per_epoch(dataset_size, b)} (dataset_size={dataset_size})")

## 3. モデル：2層MLP
非線形活性化 `ReLU` を用いた2層MLP（隠れ層1つ）を定義します。

In [ ]:
class MLP(nn.Module):
    def __init__(self, in_dim=2, hidden=32, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

model = MLP(in_dim=2, hidden=32, out_dim=2).to(DEVICE)
model

# 問3

このMLPの **総パラメータ数** を式で示し、次セルの出力を **根拠** に検算しなさい。

なお、パラメータ数は、`Linear(2→32)` の重み・バイアス、`Linear(32→2)` の重み・バイアスの合計とする。

以下にセルを追加して解答しなさい。

In [ ]:
def count_params(m: nn.Module):
    return sum(p.numel() for p in m.parameters())

print('Total params:', count_params(model))
for n, p in model.named_parameters():
    print(n, p.shape, p.numel())

## 4. 学習ルーチン（ベースライン）
損失は `CrossEntropyLoss`、最適化は `Adam` を用います。学習率や正則化は後続の演習で扱います。

In [ ]:
@dataclass
class TrainConfig:
    epochs: int = 50
    lr: float = 1e-2
    weight_decay: float = 0.0  # L2正則化

def train(model, train_loader, val_loader, cfg: TrainConfig):
    model = model.to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    hist = {'train_loss': [], 'val_loss': [], 'val_acc': []}
    for epoch in range(1, cfg.epochs+1):
        model.train()
        running = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            running += loss.item() * xb.size(0)
        train_loss = running / len(train_loader.dataset)

        # validation
        model.eval()
        val_running, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                logits = model(xb)
                loss = criterion(logits, yb)
                val_running += loss.item() * xb.size(0)
                pred = logits.argmax(dim=1)
                correct += (pred == yb).sum().item()
                total += yb.size(0)
        val_loss = val_running / len(val_loader.dataset)
        val_acc = correct / total

        hist['train_loss'].append(train_loss)
        hist['val_loss'].append(val_loss)
        hist['val_acc'].append(val_acc)
        if epoch % 10 == 0 or epoch == 1:
            print(f"epoch {epoch:3d}: train_loss={train_loss:.4f} val_loss={val_loss:.4f} val_acc={val_acc:.3f}")
    return hist

cfg = TrainConfig(epochs=50, lr=1e-2, weight_decay=0.0)
baseline_model = MLP().to(DEVICE)
hist = train(baseline_model, train_loader, val_loader, cfg)

In [ ]:
fig = plt.figure(figsize=(6,4))
plt.plot(hist['train_loss'], label='train_loss')
plt.plot(hist['val_loss'], label='val_loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Learning Curves')
plt.legend()
plt.show()

fig = plt.figure(figsize=(6,4))
plt.plot(hist['val_acc'], label='val_acc')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('Validation Accuracy')
plt.legend()
plt.show()

# 問4

学習曲線から **過学習の兆候** は見られるか？**根拠** として `train_loss` と `val_loss` の関係、および `val_acc` の挙動を挙げて説明しなさい。

以下にセルを追加して解答しなさい。

## 5. テストセットでの評価と混同行列
学習済みモデルの性能をテストセットで評価し、混同行列を描画します。

In [ ]:
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    all_pred, all_true = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logits = model(xb)
            pred = logits.argmax(dim=1)
            correct += (pred == yb).sum().item()
            total += yb.size(0)
            all_pred.append(pred.cpu())
            all_true.append(yb.cpu())
    acc = correct / total
    return acc, torch.cat(all_true), torch.cat(all_pred)

test_acc, y_true, y_pred = evaluate(baseline_model, test_loader)
print(f"Test Accuracy: {test_acc:.3f}")

def confusion_matrix(y_true, y_pred, num_classes=2):
    cm = torch.zeros((num_classes, num_classes), dtype=torch.int64)
    for t, p in zip(y_true, y_pred):
        cm[t, p] += 1
    return cm

cm = confusion_matrix(y_true, y_pred, num_classes=2)
print('Confusion Matrix:\n', cm)

fig = plt.figure(figsize=(4,4))
plt.imshow(cm, interpolation='nearest')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
for (i, j) in itertools.product(range(cm.size(0)), range(cm.size(1))):
    plt.text(j, i, int(cm[i, j]), ha='center', va='center')
plt.colorbar()
plt.show()

# 問5

混同行列から、どちらのクラスで誤分類が相対的に多いか、**根拠** を示して述べて説明しなさい。

以下にセルを追加して解答しなさい。

## 6. ハイパーパラメータの影響：学習率・L2・バッチサイズ
学習率（`lr`）、L2正則化（`weight_decay`）、バッチサイズの違いが性能に与える影響を簡易に比較します。

In [ ]:
def quick_train_eval(lr=1e-2, weight_decay=0.0, batch=64, epochs=30):
    # 新しいDataLoader（バッチサイズだけ変える）
    train_loader_q = DataLoader(train_ds, batch_size=batch, shuffle=True)
    val_loader_q   = DataLoader(val_ds, batch_size=batch)
    test_loader_q  = DataLoader(test_ds, batch_size=batch)
    m = MLP().to(DEVICE)
    h = train(m, train_loader_q, val_loader_q, TrainConfig(epochs=epochs, lr=lr, weight_decay=weight_decay))
    test_acc, _, _ = evaluate(m, test_loader_q)
    return h, test_acc

grid = {
    'lr': [1e-3, 1e-2, 5e-2],
    'weight_decay': [0.0, 1e-4, 1e-3],
    'batch': [32, 64]
}

results = []
for lr in grid['lr']:
    for wd in grid['weight_decay']:
        for b in grid['batch']:
            print(f"=== lr={lr}, weight_decay={wd}, batch={b} ===")
            h, acc = quick_train_eval(lr=lr, weight_decay=wd, batch=b, epochs=25)
            results.append({'lr': lr, 'weight_decay': wd, 'batch': b,
                            'best_val_acc': max(h['val_acc']), 'final_val_acc': h['val_acc'][-1],
                            'test_acc': acc})

import pandas as pd
df_results = pd.DataFrame(results)
df_results.sort_values(['test_acc','best_val_acc'], ascending=False, inplace=True)
df_results.reset_index(drop=True, inplace=True)
df_results

# 問6

上表を **根拠** に、`lr`、`weight_decay`、`batch` が **汎化性能（test_acc）** に与える影響を簡単にまとめなさい。このとき、併せて、`val_acc` の推移と整合しているかも考察しなさい。

以下にセルを追加して解答しなさい。

## 7. 決定境界の可視化
学習した分類器の決定境界を描画し、データ散布とともに確認します。

In [ ]:
def plot_decision_boundary(model, X, y, title='Decision Boundary', steps=300):
    model.eval()
    x_min, x_max = X[:,0].min()-0.5, X[:,0].max()+0.5
    y_min, y_max = X[:,1].min()-0.5, X[:,1].max()+0.5
    xx = torch.linspace(x_min, x_max, steps)
    yy = torch.linspace(y_min, y_max, steps)
    grid = torch.stack(torch.meshgrid(xx, yy, indexing='xy'), dim=-1).reshape(-1, 2)
    with torch.no_grad():
        logits = model(grid.to(DEVICE))
        pred = logits.argmax(dim=1).cpu().reshape(steps, steps)
    fig = plt.figure(figsize=(5,5))
    plt.contourf(xx, yy, pred, alpha=0.3)
    plt.scatter(X[:,0], X[:,1], c=y, s=8)
    plt.title(title)
    plt.xlabel('x1'); plt.ylabel('x2')
    plt.show()

plot_decision_boundary(baseline_model, X_test, y_test, title='Baseline Decision Boundary (Test)')

### Exercise 7（観察）
- 決定境界の形状を観察し、**非線形性** の必要性について簡潔に述べてください。

_以下に解答を書く_

## 8. 保存と復元（`state_dict`）
学習済みモデルを保存し、復元して再評価します。

In [ ]:
save_path = 'baseline_mlp_state.pt'
torch.save(baseline_model.state_dict(), save_path)
print('saved to', save_path)

loaded = MLP().to(DEVICE)
loaded.load_state_dict(torch.load(save_path, map_location=DEVICE))
acc_loaded, _, _ = evaluate(loaded, test_loader)
print('Reloaded Test Accuracy:', acc_loaded)

#問8

復元後の精度は保存前と一致しているか、**根拠** として数値を示し、差が出る可能性があるケース（例えばBatchNormなど状態を持つ層を導入した場合）も言及しなさい。

以下にセルを追加して解答しなさい。

## 9. 再現性とシード
乱数シードを変えると初期値が変わり、性能がぶれることがあります。軽く検証します。

In [ ]:
def run_with_seed(seed):
    set_seed(seed)
    m = MLP().to(DEVICE)
    h = train(m, train_loader, val_loader, TrainConfig(epochs=30, lr=1e-2))
    acc, _, _ = evaluate(m, test_loader)
    return max(h['val_acc']), acc

seeds = [2023, 2024, 2025]
records = []
for s in seeds:
    best_val, test_acc = run_with_seed(s)
    records.append({'seed': s, 'best_val_acc': best_val, 'test_acc': test_acc})
import pandas as pd
df_seeds = pd.DataFrame(records)
df_seeds

# 問9

シードによる性能のばらつきを **表（上セル）** を根拠に一言でまとめ、再現性確保のための一般的な実務上の工夫（種の固定、実験回数、平均と標準偏差の報告など）について述べなさい。

以下にセルを追加して解答しなさい。

# 問10

活性化関数を `Tanh`/`LeakyReLU` に変更し、**可視化や表** を用いて **根拠** を明示しつつ、その効果について説明しなさい。